In [2]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time`
import torch

import pymorphy2
import re
import spacy

from string import punctuation
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch

data_path = "data/"

import os
print(os.cpu_count())

64


In [3]:
df = pd.read_parquet(data_path+"labeled/1090_df_labeled.parquet.gzip")
df.shape

(541981, 15)

In [ ]:
!ls -a data

In [58]:
df[df.group.apply(lambda v: v.split("_")[1]) != "-1"].to_parquet("data/test.csv.gzip", index=False, compression="gzip")

In [46]:
df[df.group == np.random.choice(df.group.unique())][["title", "description", "group"]]

,title,description,group
367214,LAN кабель для интернета cat5e RJ45 1Гбит патч...,Интернетный кабель высокого качества. Патч кор...,9963_20
367297,LAN кабель для интернета cat5e RJ45 1Гбит патч...,Интернетный кабель высокого качества. Патч кор...,9963_20
367323,LAN кабель для подключения интернет патч корд ...,Интернетный кабель высокого качества. Патч кор...,9963_20
367463,LAN кабель для интернета cat5e RJ45 1Гбит патч...,Интернетный кабель высокого качества. Патч кор...,9963_20
367591,LAN кабель для интернета cat5e RJ45 1Гбит патч...,Интернетный кабель высокого качества. Патч кор...,9963_20


In [50]:
cnt_target = df.group.value_counts()
print(len(cnt_target[cnt_target < 2].keys()))

# drop trash group
df_good = df[~df.group.apply(lambda v: v.split("_")[1]).isin(["-1"])].reset_index(drop=True)
# drop single targets
df_good = df[~df.group.isin(list(cnt_target[cnt_target < 2].keys()))].reset_index(drop=True)

print(df.shape, df_good.group.nunique())

1
(141427, 15) 9140


# histogram

In [64]:
st = time.time()

clf = HistGradientBoostingClassifier(max_iter=1000)
clf.fit(
    np.array(list(df_good.embeddings), dtype=float),
    df_good.group.values
)

print(f"time: {time.time() - st}")

time: 324.80134177207947


In [65]:
df_sample = df_good[["embeddings", "group"]].sample(5)
df_sample

,embeddings,group
58593,"[0.019851577, -0.014928423, -0.015613066, -0.0...",967
18499,"[0.011278529, -0.009696034, -0.0001188231, -0....",1380
21729,"[0.008027035, -0.025474597, 0.0033335777, -0.0...",1768
6585,"[0.034189533, -0.014423374, -0.023568464, -0.0...",430
54366,"[0.026823383, -0.016151477, -0.014018539, -0.0...",1917


In [66]:
clf.predict(np.array(list(df_sample.embeddings), dtype=float))

array([ 72, 299, 520, 775,  93])

# torch

In [7]:
if False:
    df_good.group = df_good.group - 1

In [51]:
label_mapping = {label: idx for idx, label in enumerate(set(df_good.group.values))}

df_good.group = df_good.group.map(label_mapping)

In [52]:
xtr, xts, ytr, yts = train_test_split(
    np.array(list(df_good.embeddings), dtype=float),
    df_good.group.values,
    test_size = 0.2, random_state=42
)
print(xtr.shape, xts.shape, ytr.shape, yts.shape)

xtr = torch.tensor(xtr, dtype=torch.float32)
xts = torch.tensor(xts, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.long)
yts = torch.tensor(yts, dtype=torch.long)

(113140, 1024) (28286, 1024) (113140,) (28286,)


In [57]:
n_output = df_good.group.nunique()
print(n_output)

9140


In [ ]:
class DeepClassifier(torch.nn.Module):
    def __init__(self, n_output):
        super(DeepClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(xtr.shape[1], 20480)
        self.fc2 = torch.nn.Linear(20480, 16384)
        self.fc3 = torch.nn.Linear(16384, 12288)
        self.fc4 = torch.nn.Linear(12288, n_output+1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
    
model = DeepClassifier(n_output)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
batch_size = 64
num_batches = len(xtr) // batch_size

for epoch in range(num_epochs):
    model.train()
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        batch_x = xtr[batch_start:batch_end]
        batch_y = ytr[batch_start:batch_end]
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        test_outputs = model(xts)
        test_preds = torch.argmax(test_outputs, dim=1)
        test_accuracy = accuracy_score(yts, test_preds.numpy())
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.4f}')
        with open(data_path+"logs_classifier.txt", "a") as f:
            f.write(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.4f}\n')

Epoch [1/10], Test Accuracy: 0.2681
Epoch [2/10], Test Accuracy: 0.2857
Epoch [3/10], Test Accuracy: 0.3125
Epoch [5/10], Test Accuracy: 0.3613
Epoch [6/10], Test Accuracy: 0.3794
Epoch [7/10], Test Accuracy: 0.3886
Epoch [8/10], Test Accuracy: 0.3997


In [60]:
print(1)

1
